[DQN explanation](https://towardsdatascience.com/deep-q-network-dqn-i-bce08bdf2af)

In [7]:
import gym
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

print('current path: {}'.format(os.getcwd()))

current path: C:\Users\DALab\Documents\GitHub\pytorch_learning\Reinforcement Learning


In [8]:
env = gym.make('CartPole-v0').unwrapped

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [9]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

### Deep Q-Network

In [11]:
class DQN(nn.Module):

    def __init__(self, h, w, outputs):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)

        # Number of Linear input connections depends on output of conv2d layers
        # and therefore the input image size, so compute it.
        def conv2d_size_out(size, kernel_size = 5, stride = 2):
            return (size - (kernel_size - 1) - 1) // stride  + 1
        convw = conv2d_size_out(conv2d_size_out(conv2d_size_out(w)))
        convh = conv2d_size_out(conv2d_size_out(conv2d_size_out(h)))
        linear_input_size = convw * convh * 32
        self.head = nn.Linear(linear_input_size, outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))

### Input Extraction

In [10]:
resize = T.Compose([T.ToPILImage(),
                    T.Resize(40, interpolation=Image.CUBIC),
                    T.ToTensor()])


def get_cart_location(screen_width):
    world_width = env.x_threshold * 2
    scale = screen_width / world_width
    return int(env.state[0] * scale + screen_width / 2.0)  # MIDDLE OF CART

def get_screen():
    # Returned screen requested by gym is 400x600x3, but is sometimes larger
    # such as 800x1200x3. Transpose it into torch order (CHW).
    screen = env.render(mode='rgb_array').transpose((2, 0, 1))
    # Cart is in the lower half, so strip off the top and bottom of the screen
    _, screen_height, screen_width = screen.shape
    screen = screen[:, int(screen_height*0.4):int(screen_height * 0.8)]
    view_width = int(screen_width * 0.6)
    cart_location = get_cart_location(screen_width)
    if cart_location < view_width // 2:
        slice_range = slice(view_width)
    elif cart_location > (screen_width - view_width // 2):
        slice_range = slice(-view_width, None)
    else:
        slice_range = slice(cart_location - view_width // 2,
                            cart_location + view_width // 2)
    # Strip off the edges, so that we have a square image centered on a cart
    screen = screen[:, :, slice_range]
    # Convert to float, rescale, convert to torch tensor
    # (this doesn't require a copy)
    screen = np.ascontiguousarray(screen, dtype=np.float32) / 255
    screen = torch.from_numpy(screen)
    # Resize, and add a batch dimension (BCHW)
    return resize(screen).unsqueeze(0)


env.reset()
plt.figure()
plt.imshow(get_screen().cpu().squeeze(0).permute(1, 2, 0).numpy(),
           interpolation='none')
plt.title('Example extracted screen')
plt.show()

C:\Users\DALab\anaconda3\envs\dl_test\lib\site-packages\torchvision\transforms\transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


ImportError: 
    Cannot import pyglet.
    HINT: you can install pyglet directly via 'pip install pyglet'.
    But if you really just want to install all Gym dependencies and not have to think about it,
    'pip install -e .[all]' or 'pip install gym[all]' will do it.
    

<Figure size 432x288 with 0 Axes>

In [72]:
from torchvision import transforms  

def make_env(env_name, mode='train'):
    env = gym.make(env_name)
    env = FrameStack(env)
    # different reward
    if mode == 'train':
        env = SurvivalRewardEnv(env)
    return env

def image_processing(state, num_frame=4):

    print('print state shape: ', state.shape)
            
    transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize(size=(84,84)),
                    transforms.Grayscale(num_output_channels=1),
                    transforms.ToTensor()
                ])
    
    for i in range(state.shape[-1]//3):
        image = state[:,:, i*3:i*3+3]
        transformed_image = transform(image)
        
        if i == 0:
            stack_frames = transformed_image
        else:
            stack_frames = torch.cat((stack_frames, transformed_image), axis=0)
        
    print('stack frame shape: ', stack_frames.shape)
    return stack_frames

In [73]:
a = np.random.rand(84, 168, 12).astype(np.uint8)
stack_frames = image_processing(a)


print state shape:  (84, 168, 12)
stack frame shape:  torch.Size([4, 84, 84])
torch.Size([4, 84, 84])


In [ ]:
torch

In [ ]:
def image_processing(state):
    
    transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((84,84)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
    ])
    new_img = transform(state)
    return new_img

In [ ]:
model.train()

BN(8)